# **Exploratory Data Analisys: Companies**

## **Initial Setup**

### Install Packages

In [1]:
%pip install pandas -q
%pip install plotly -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import libs

In [38]:
import os
import itertools
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import warnings
warnings.filterwarnings('ignore')

### Pandas Config

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Create a file path default

In [4]:
file_path_book = str(Path(os.getcwd()).parent.parent.parent / "data/book")

## Macroeconomic

### Load data

In [5]:
df_macroeconomic_book = pd.read_csv(file_path_book + "/macroeconomic_book.csv")
df_macroeconomic_book.head(5)

,date,selic,confidence,pib,incc,ipca,dolar,monthly_inflation,gdp_growth,dollar_growth,real_interest_rate,inflation_confidence_difference
0,2019-01-31,6.5,128.64,578214.5,0.49,3.78,3.6513,-0.111948,-0.015756,0.016459,2.72,-124.86
1,2019-02-28,6.5,139.39,576089.7,0.09,3.89,3.7379,0.029101,-0.003675,0.023718,2.61,-135.50
2,2019-03-31,6.5,125.53,601749.8,0.31,4.58,3.8961,0.177378,0.044542,0.042323,1.92,-120.95
3,2019-04-30,6.5,121.71,612918.4,0.38,4.94,3.9447,0.078603,0.018560,0.012474,1.56,-116.77
4,2019-05-31,6.5,117.01,615304.9,0.03,4.66,3.9401,-0.056680,0.003894,-0.001166,1.84,-112.35


In [6]:
df_macroeconomic_numeric_cols = df_macroeconomic_book.select_dtypes(include=["int", "number", "float64"])
df_macroeconomic_numeric_cols.head(5)

,selic,confidence,pib,incc,ipca,dolar,monthly_inflation,gdp_growth,dollar_growth,real_interest_rate,inflation_confidence_difference
0,6.5,128.64,578214.5,0.49,3.78,3.6513,-0.111948,-0.015756,0.016459,2.72,-124.86
1,6.5,139.39,576089.7,0.09,3.89,3.7379,0.029101,-0.003675,0.023718,2.61,-135.50
2,6.5,125.53,601749.8,0.31,4.58,3.8961,0.177378,0.044542,0.042323,1.92,-120.95
3,6.5,121.71,612918.4,0.38,4.94,3.9447,0.078603,0.018560,0.012474,1.56,-116.77
4,6.5,117.01,615304.9,0.03,4.66,3.9401,-0.056680,0.003894,-0.001166,1.84,-112.35


### Macroeconomic Analisys

#### Coefficient of variation

In [37]:
df_macroeconomic_book_cvs = ((df_macroeconomic_numeric_cols.std(ddof=1) / df_macroeconomic_numeric_cols.mean().sort_values(ascending=True)) * 100).sort_values(ascending=True).reset_index()
df_macroeconomic_book_cvs.columns = ['column', 'coefficient_variation']
df_macroeconomic_book_cvs['coefficient_variation'] = df_macroeconomic_book_cvs['coefficient_variation'].abs().sort_values(ascending=True)
df_macroeconomic_book_cvs.fillna(method='ffill', inplace=True)
df_macroeconomic_book_cvs = df_macroeconomic_book_cvs.sort_values(by='coefficient_variation', ascending=False)


fig = px.bar(df_macroeconomic_book_cvs, title='Coefficient of Variation (%)',y='column', x='coefficient_variation', orientation='h', color_discrete_sequence=['rgb(100, 195, 181)'],hover_name='column', height=800)
fig.update_traces(text=[f'{x:.0f}%' for x in df_macroeconomic_book_cvs['coefficient_variation']], textposition='outside')
fig.update_layout(xaxis_title='Coefficient of Variation (%)', yaxis_title='Columns', template='plotly_dark', font=dict(color='white'), xaxis={'type':'log', 'title':'Coefficient of Variation (Log Scale)'}, hovermode='y')

fig.show()

/var/folders/sl/456ghqp95x1d0ly3kxqstmsm0000gn/T/ipykernel_2213/3593683114.py:4: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



#### Outlier Analysis

In [7]:
num_columns = len(df_macroeconomic_book)
num_rows = (num_columns + 2) // 2

subplot_titles = [str(col) for col in df_macroeconomic_book]

fig = sp.make_subplots(rows=num_rows, cols=3, subplot_titles=subplot_titles)

for i, column in enumerate(df_macroeconomic_book, start=1):
    row = (i - 1) // 3 + 1
    col = (i - 1) % 3 + 1

    trace = go.Box(y=df_macroeconomic_book[column], name=column, 
                   marker_color='lightseagreen', boxpoints='outliers', 
                   jitter=0.7, hoverinfo='y+text', 
                   text=())

    fig.add_trace(trace, row=row, col=col)

fig.update_layout(title_text='Boxplot of Numerical Variables', height=300*num_rows, showlegend=False, template='plotly_dark')
fig.show()

### Histogram

In [8]:
subplot_titles = [str(col) for col in df_macroeconomic_book]
columns_per_row = 3

num_rows = len(df_macroeconomic_book) // columns_per_row + (len(df_macroeconomic_book) % columns_per_row > 0)
fig = sp.make_subplots(rows=num_rows, cols=columns_per_row, subplot_titles=subplot_titles)

for i, column in enumerate(df_macroeconomic_book):
    row = i // columns_per_row + 1
    col = i % columns_per_row + 1
    
    fig.add_trace(go.Histogram(x=df_macroeconomic_book[column], name=column, marker_color='lightseagreen'), row=row, col=col)

fig.update_layout(title='Histograms by Column', height=300 * num_rows, showlegend=False, template='plotly_dark')
fig.show()

### Correlation and Dispersion

In [9]:

df_macroeconomic_numeric_cols_corr = df_macroeconomic_numeric_cols.corr()
color_scale = [[0, 'rgb(150, 245, 231)'], [0.5, 'rgb(100, 195, 181)'], [1, 'rgb(50, 145, 131)']]
fig = px.imshow(df_macroeconomic_numeric_cols_corr, text_auto=True, aspect="auto", width=1000, height=500, template="plotly_dark", title="Mapa de Calor: Correlação entre as variaveis", color_continuous_scale=color_scale)
fig.show()

In [10]:
fig = px.scatter_matrix(df_macroeconomic_numeric_cols_corr, dimensions=df_macroeconomic_numeric_cols_corr.columns, title='Gráficos de Dispersão para Todas as Colunas', template="plotly_dark",)
fig.update_traces(marker=dict(color='rgb(100, 195, 181)'))

fig.update_layout(width=4000, height=3000, grid=dict(xgap=0.1, ygap=0.5))

fig.show()

### Análise de Tendência

#### SELIC and Real Interest Rate

In [95]:
fig_selic = sp.make_subplots(specs=[[{"secondary_y": True}]])

fig_selic.add_trace(go.Scatter(x=df_macroeconomic_book['date'], y=df_macroeconomic_book['selic'], name='SELIC', line=dict(color='rgb(100, 195, 181)')), secondary_y=False)
fig_selic.add_trace(go.Scatter(x=df_macroeconomic_book['date'], y=df_macroeconomic_book['real_interest_rate'], name='Real Interest Rate', line=dict(color='rgb(255, 0, 0)')), secondary_y=True)

fig_selic.update_layout(title_text='SELIC Rate and Real Interest Rate Over Time', template='plotly_dark', legend=dict(font=dict(color='white')), height=600)

fig_selic.update_xaxes(title='Date', title_font=dict(color='white'), tickfont=dict(color='white'))
fig_selic.update_yaxes(title='SELIC Rate (%)', secondary_y=False, title_font=dict(color='white'), tickfont=dict(color='white'))
fig_selic.update_yaxes(title='Real Interest Rate (%)', secondary_y=True, title_font=dict(color='white'), tickfont=dict(color='white'))

fig_selic.show()

The graph illustrates the trajectory of the `SELIC rate` and the `Real Interest Rate` from `July 2019` to `July 2023`, highlighting key economic indicators that influence Brazil's monetary policy.

**Economic Context (`2019-2020`)**

- **July 2019**: The `SELIC rate` was around **6%**, with the `Real Interest Rate` slightly above **2%**.
- **2020**: `SELIC` experienced a gradual decline, reaching near **2%** by mid-2020. The `Real Interest Rate` dipped into negative territory, suggesting real earnings on savings were below inflation.

**Policy Shift (`2021`)**

- **Early 2021**: `SELIC` started to increase, indicating tightening monetary policy, moving from **2%** to above **6%** by year-end. The `Real Interest Rate` also began to rise from negative values, reflecting an adjustment to rising inflation.

**Inflation Control Measures (`2022`)**

- **Early 2022**: `SELIC` reached a peak above **12%**, while the `Real Interest Rate` climbed to around **6%**, showing the Central Bank's aggressive measures to control inflation.

**Economic Stabilization (`Mid-2022 to 2023`)**

- **Mid-2022 to Early 2023**: The `SELIC rate` plateaued at around **12%**, with the `Real Interest Rate` stabilizing around **5-6%**. This indicates an attempt to maintain a tight grip on inflation while offering real growth in savings.

**Recent Trends (`Mid-2023`)**

- **July 2023**: The `SELIC rate` shows a slight decrease to just below **12%**, while the `Real Interest Rate` has a modest downtrend, yet remains positive, suggesting a cautious yet slightly relaxed monetary policy stance.

The graph reflects the Central Bank's response to economic cycles and inflationary pressures, adjusting the `SELIC rate` to manage inflation and maintain positive `Real Interest Rates` for savers.


#### Consumer Confidence:

In [99]:
fig_confidence = px.line(df_macroeconomic_book, x='date', y='confidence', title='Consumer Confidence', color_discrete_sequence=['rgb(100, 195, 181)'], template='plotly_dark')
fig_confidence.update_xaxes(title='Date', title_font=dict(color='white'), tickfont=dict(color='white'))
fig_confidence.update_yaxes(title='Confidence Index', title_font=dict(color='white'), tickfont=dict(color='white'))

fig_confidence.show()

The graph tracks the changes in the `Consumer Confidence Index` over a four-year period, reflecting the public's economic sentiment.

**Initial Confidence (`July 2019`)**

- At the start of the period, the index stood at around **110**, suggesting moderate consumer confidence.

**Peak and Trough (`2020`)**

- The index peaked at over **130** in early `2020`, indicating a strong consumer outlook.
- It then fell sharply, bottoming out near **100** by mid-`2020`, likely due to economic uncertainties.

**Gradual Recovery (`2021`)**

- Throughout `2021`, the index showed a gradual recovery from the previous low, hovering around **110-120**, signaling improving consumer sentiment.

**Fluctuations and Recovery (`2022`)**

- `2022` saw fluctuations with the index oscillating between **110** and just above **120**.
- By the end of `2022`, the index had an upward trend, indicating a return of consumer optimism.

**Recent Uptrend (`2023`)**

- As of `July 2023`, the index has climbed to approximately **130**, a sign of robust consumer confidence and positive economic expectations.

The Consumer Confidence Index's trajectory over these years shows the volatility of consumer sentiment in response to economic changes and potential recovery perceptions in recent months.


#### (PIB) GDP growth (Gross Domestic Product growth):

In [100]:
fig_gdp = px.line(df_macroeconomic_book, x='date', y='gdp_growth', title='GDP Growth Rate', color_discrete_sequence=['rgb(100, 195, 181)'], template='plotly_dark')
fig_gdp.update_xaxes(title='Date', title_font=dict(color='white'), tickfont=dict(color='white'))
fig_gdp.update_yaxes(title='GDP Growth Rate (%)', title_font=dict(color='white'), tickfont=dict(color='white'))

fig_gdp.show()

The graph depicts the fluctuations in the `GDP Growth Rate`, an important economic indicator reflecting the health of the economy over time.

**Oscillating Growth (`2019-2020`)**

- Starting in `July 2019`, the growth rate fluctuated around **0%**, indicating a relatively stagnant economy.
- In `2020`, the rate experienced a sharp decline, reaching a trough of nearly **-0.1%**, suggesting a significant economic contraction.

**Recovery and Instability (`2021`)**

- The year `2021` showed signs of recovery with rates rising above **0%**, but still displayed considerable instability with sharp ups and downs.

**Volatile Recovery (`2022`)**

- Throughout `2022`, the growth rate continued to demonstrate volatility, peaking at just above **0.1%** and dipping below **0%** multiple times.

**Recent Trends (`2023`)**

- As of `July 2023`, the `GDP Growth Rate` appears to be stabilizing with positive growth just under **0.05%**, potentially signaling a more consistent economic recovery.

The `GDP Growth Rate` graph illustrates the challenges and resilience of the economy, with recent trends suggesting a gradual return to steady growth.


#### Dólar e Inflação Mensal:

In [101]:
fig_dollar_inflation = sp.make_subplots(specs=[[{"secondary_y": True}]])

fig_dollar_inflation.add_trace(go.Scatter(x=df_macroeconomic_book['date'], y=df_macroeconomic_book['dolar'], name='Dollar Exchange Rate', line=dict(color='rgb(100, 195, 181)')), secondary_y=False)
fig_dollar_inflation.add_trace(go.Scatter(x=df_macroeconomic_book['date'], y=df_macroeconomic_book['monthly_inflation'], name='Monthly Inflation', line=dict(color='rgb(255, 0, 0)')), secondary_y=True,)
fig_dollar_inflation.update_layout(title_text='Dollar Exchange Rate and Monthly Inflation Over Time', template='plotly_dark', legend=dict(font=dict(color='white')))

fig_dollar_inflation.update_xaxes(title='Date', title_font=dict(color='white'), tickfont=dict(color='white'))
fig_dollar_inflation.update_yaxes(title='Dollar Exchange Rate (BRL/USD)', secondary_y=False, title_font=dict(color='white'), tickfont=dict(color='white'))
fig_dollar_inflation.update_yaxes(title='Monthly Inflation (%)', secondary_y=True, title_font=dict(color='white'), tickfont=dict(color='white'))

fig_dollar_inflation.show()

This graph presents the interrelation between the `Dollar Exchange Rate` and `Monthly Inflation`, offering insights into their impact on economic dynamics.

**Exchange Rate Volatility (`2019-2020`)**

- **July 2019**: The exchange rate started at around **4 BRL/USD**.
- **2020**: It saw significant volatility, peaking at over **5.5 BRL/USD** and correlating with substantial economic uncertainty.

**Inflation Fluctuations (`2020-2021`)**

- During `2020` and into `2021`, `Monthly Inflation` also fluctuated, with rates oscillating between **-0.2%** and **0.3%** monthly, reflecting the reactive nature of prices to external and internal economic shocks.

**Stabilizing Trend (`2021-2022`)**

- **2021 to 2022**: The `Dollar Exchange Rate` appeared to stabilize somewhat, fluctuating around **5 BRL/USD**.
- Concurrently, `Monthly Inflation` showed a dampening of volatility, with most values ranging between **0%** and **0.2%**.

**Recent Developments (`2023`)**

- **January to July 2023**: The exchange rate experienced a downward trend, approaching **4.5 BRL/USD**.
- `Monthly Inflation` saw a spike above **0.2%**, potentially signaling emerging inflationary pressures or a response to changes in the exchange rate.

The graph highlights the complex relationship between currency exchange rates and inflation, with the former influencing import costs and the latter reflecting the cost of living adjustments.

### Volatility Analysis:

In [104]:
window_size = 2

df_macroeconomic_book['rolling_std_dollar'] = df_macroeconomic_book['dolar'].rolling(window=window_size).std()
df_macroeconomic_book['rolling_std_inflation'] = df_macroeconomic_book['ipca'].rolling(window=window_size).std()

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_macroeconomic_book['date'], y=df_macroeconomic_book['rolling_std_dollar'], name='Rolling Std Dev - Dollar', line=dict(color='rgb(100, 195, 181)')))
fig.update_layout(title='Volatility of Dollar Exchange Rate (Rolling Standard Deviation)', xaxis_title='Date', yaxis_title='Rolling Std Dev', template='plotly_dark')

fig.add_trace(go.Scatter(x=df_macroeconomic_book['date'], y=df_macroeconomic_book['rolling_std_inflation'], name='Rolling Std Dev - Inflation', line=dict(color='tomato', width=2)))
fig.update_layout(title='Volatility of Monthly Inflation (Rolling Standard Deviation)', xaxis_title='Date', yaxis_title='Rolling Std Dev', template='plotly_dark')

fig.show()

The graph depicts the volatility, as measured by the `Rolling Standard Deviation`, for both the `Dollar Exchange Rate` and `Monthly Inflation` from `July 2019` to `July 2023`.

**Observations**"

- There are notable spikes in the rolling standard deviation for both metrics, with the `Dollar Exchange Rate` generally exhibiting higher volatility.
- The peak volatility for `Monthly Inflation` occurs around the start of `2022`, with a rolling standard deviation nearing **1.2**.
- The `Dollar Exchange Rate` shows several peaks, with one of the highest around `September 2020`, reaching approximately **0.5**.
- As of `July 2023`, the rolling standard deviation for both the `Dollar Exchange Rate` and `Monthly Inflation` shows a downtrend, indicating a period of relative economic stability.

**Interpretation**:

- The higher peaks in rolling standard deviation for the `Dollar Exchange Rate` suggest more pronounced fluctuations in value, potentially due to market reactions to global economic events.
- The `Monthly Inflation` volatility peaks might correspond to periods of economic policy changes or external economic shocks.
- The recent trends towards lower volatility may suggest that the market conditions are stabilizing, or that the measures to control inflation and exchange rate fluctuations are taking effect.

This data can be instrumental for policymakers and investors to understand the economic environment's stability and to make informed decisions.

### Technical Analysis:

In [107]:
fig_ipca_bar = px.bar(df_macroeconomic_book, x='date', y='ipca', title='Monthly IPCA Over Time', labels={'ipca': 'IPCA (%)'})
fig_ipca_bar.update_layout(template='plotly_dark', xaxis_title='Date', yaxis_title='IPCA (%)')

fig_ipca_bar.show()

This bar graph represents the `Monthly IPCA` rate, which is a key indicator of inflation in the Brazilian economy.

**Trend Analysis**:

- The IPCA starts at a moderate level in `July 2019`, with a percentage just above **3%**.
- A noticeable trend is the gradual increase in IPCA starting from early `2020`, reaching a peak around `January 2022` with an IPCA close to **11%**.
- After the peak in `January 2022`, there's a downward trend, with the IPCA rate decreasing to levels around **6%** by `July 2023`.

**Economic Implications**:

- The steady rise in IPCA through `2020` and `2021` could indicate mounting inflationary pressures within the economy.
- The peak in `January 2022` may reflect the culmination of various economic factors, potentially including supply chain disruptions or changes in fiscal policy.
- The subsequent decline suggests that measures to control inflation may be taking effect, or that the economy is adjusting to the previous pressures.

The IPCA trends shown in this graph are critical for understanding the inflationary landscape in Brazil and can impact both monetary policy decisions and investment strategies.


In [108]:
fig_inflation_trend = px.line(df_macroeconomic_book, x='date', y='ipca', title='Monthly Inflation Trend', labels={'ipca': 'Monthly Inflation (%)'})

fig_inflation_trend.update_layout(template='plotly_dark', xaxis_title='Date', yaxis_title='Monthly Inflation (%)')

fig_inflation_trend.show()

**Detailed Observations**

- The inflation rate begins modestly in `July 2019`, averaging around **3.5%**.
- There is a noticeable dip and subsequent recovery during `2020`, with the lowest point close to **2%** and a rebound towards **3%** by the end of the year.
- A significant upward trend starts from early `2021`, with the rate steadily climbing and peaking at just above **10%** in `January 2022`.
- Post-peak, the trend reverses with a sharp decline to approximately **6%** by mid-`2022`.
- The trend flattens out towards the end of the graph, but a slight uptick is observed in `August 2023`, with the rate at **4.61%**.

**Interpretation and Implications**

- The initial stability followed by a dip in `2020` could be associated with economic responses to global events such as the pandemic.
- The inflation spike through `2021` into early `2022` suggests significant inflationary pressures, possibly exacerbated by recovery measures and supply chain challenges.
- The reduction in inflation rate post-`January 2022` might indicate the effectiveness of policy measures or an adjustment phase of the economy to the previous inflationary conditions.
- The recent stabilization and slight increase as of `August 2023` could signal a cautious period of economic activity, with inflationary pressures being closely monitored.

This inflation trend is a vital indicator for economic health, influencing consumer spending, business investment, and policy-making decisions.


### Inflation

In [130]:

df = df_macroeconomic_book.copy(deep=True)
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df_annual_inflation = df.groupby('year')['ipca'].mean().reset_index()

fig = sp.make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=df['date'], y=df['ipca'], name='Monthly Inflation'), secondary_y=False)
fig.add_trace(go.Scatter(x=df_annual_inflation['year'], y=df_annual_inflation['ipca'], name='Annual Inflation', marker=dict(color='red')), secondary_y=True)

fig.update_layout(title_text='Monthly and Annual Inflation Comparison', template='plotly_dark')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Monthly Inflation (%)', secondary_y=False)
fig.update_yaxes(title_text='Annual Inflation (%)', secondary_y=True)


fig.show()

The combined bar and line graph provides a nuanced view of the inflationary trends over a period spanning from `January 2019` to `July 2023`.

**Monthly Inflation Rate**

- The rate begins at a modest level in `January 2019`, with the `Monthly Inflation` at **2.31%**.
- A gradual uptick is observed in `2020`, with a notable increase starting in `May 2020` at **1.88%** as the monthly rate climbs past **10%**.
- The `Monthly Inflation` peaks in `April 2022`, reaching the highest point at around **12%**.
- Following this peak, there is a significant downward trend, with rates decreasing consistently to about **4%** by `June 2023`.

**Annual Inflation Trend**

- The `Annual Inflation` starts off above **4%** in `January 2019` and remains stable for several months.
- A steady increase in `Annual Inflation` is evident from mid-`2020`, with the trend line showing a gradual climb.
- This trend reaches its zenith in `January 2022`, with the `Annual Inflation` rate nearing **10%**.
- Subsequent to the peak, the `Annual Inflation` exhibits a decline, reaching levels close to **4%** by `July 2023`.

**Comparative Insights**

- The comparison between the two rates indicates that monthly fluctuations are more pronounced, while the annual rate provides a smoother long-term trend.
- The peaks and valleys in the `Monthly Inflation` rate suggest more immediate responses to short-term economic changes, while the `Annual Inflation` reflects longer-term economic trends.

This graph serves as an important tool for economic analysis, highlighting the distinct behaviors of short-term and long-term inflation measures and providing a comprehensive view of the inflationary patterns over the past


#### Monetary Policies in Inflation

In [132]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_macroeconomic_book['date'], y=df_macroeconomic_book['selic'], name="SELIC", mode='lines', line=dict(color='rgb(100, 195, 181)')))
fig.add_trace(go.Scatter(x=df_macroeconomic_book['date'], y=df_macroeconomic_book['ipca'], name="IPCA", mode='lines', line=dict(color='rgb(200, 0, 0)')))

fig.update_layout(title_text="Impact of Monetary Policy on Inflation", template='plotly_dark', xaxis_title='Date', yaxis_title='Rates (%)')

fig.show()

The SELIC and IPCA rates from mid-2019 to mid-2023 tell a story of economic shifts and monetary policy responses. Below is the numerical data contextualized within each phase of this period.

**`Economic Context` (2019-2020)**:

- **2019**: Both SELIC and IPCA started at approximately **6%**, indicating a stable economic backdrop.
- **2020**: SELIC decreased to about **4%**, a move likely aimed at economic stimulation. IPCA correspondingly fell to roughly **2%**, showing reduced inflation during a global economic downturn.

**`Policy Response` (2020-2021)**
- **2020 to 2021**: SELIC started a gradual climb from about **4%**, as the Central Bank anticipated inflationary pressures. IPCA also rose from its low, suggesting a recovering economy and associated inflationary trends.

**`Inflation Peak` (Early 2022)**
- **2022**: SELIC approached **11%**, in response to IPCA peaking over **12%**. This reflects the reactive nature of monetary policy to inflationary spikes.

**`Stabilization Efforts` (Mid-2022 to Early 2023)**
- **Mid-2022**: SELIC plateaued near **11%**, while IPCA retracted to approximately **6%**. This period marks the central bank's efforts to stabilize the economy and control inflation.

**`Forward-Looking Indicators` (Beyond Mid-2023)**
- **Early 2023**: SELIC remained steady from its plateau. IPCA experienced a minor uptick to about **7%**, signaling the need for continued vigilance in monetary policy.

The data presents a narrative of careful calibration by the Central Bank to the evolving economic landscape, with adjustments in the SELIC rate closely tracking shifts in inflation as measured by the IPCA.
